<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/agi_with_eval_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision tqdm matplotlib tensorboard

In [ ]:
#!/usr/bin/env python3
"""
agi_with_eval.py

– Synthetic train/val split
– SelfRecursiveAGI with inner/outer loops + Dropout
– Training loop with checkpoint & eval every `eval_interval` epochs
– TensorBoard logging (scalars & figures)
– Matplotlib plots saved under `out/`
– Uses parse_known_args() to ignore Jupyter’s extra CLI flags
"""

import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from tqdm import tqdm


# -----------------------------------------------------------------------------
# 1. Data
# -----------------------------------------------------------------------------
class SyntheticRegressionDataset(Dataset):
    def __init__(self, num_samples=2500, input_size=10, val_split=500):
        X = torch.randn(num_samples, input_size)
        true_w = torch.randn(input_size, 1)
        y = X @ true_w + 0.1 * torch.randn(num_samples, 1)
        data = list(zip(X, y))
        train_len = num_samples - val_split
        self.train_set, self.val_set = random_split(data, [train_len, val_split])

    def get_loaders(self, batch_size=32):
        train_loader = DataLoader(self.train_set, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(self.val_set,   batch_size=batch_size, shuffle=False)
        return train_loader, val_loader


# -----------------------------------------------------------------------------
# 2. Model
# -----------------------------------------------------------------------------
class SelfRecursiveAGI(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,
                 lr_main=1e-3, lr_self=1e-3, dropout_p=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(hidden_size, output_size)
        )
        self.main_opt = optim.Adam(self.parameters(), lr=lr_main)
        self.self_opt = optim.Adam(self.parameters(), lr=lr_self)

    def forward(self, x):
        return self.net(x)

    def self_improve(self, loss_fn, x, y, steps=5, clip=1.0):
        last = 0.0
        for _ in range(steps):
            self.self_opt.zero_grad()
            loss = loss_fn(self(x), y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.parameters(), clip)
            self.self_opt.step()
            last = loss.item()
        return last

    @torch.no_grad()
    def mc_predict(self, x, mc_samples=50):
        self.train()  # keep dropout active
        preds = []
        for _ in range(mc_samples):
            preds.append(self(x).cpu().numpy())
        arr = np.stack(preds, 0)  # [T, batch, 1]
        return arr.mean(0).squeeze(), arr.std(0).squeeze()


# -----------------------------------------------------------------------------
# 3. Training + Evaluation
# -----------------------------------------------------------------------------
def train_and_evaluate(args):
    os.makedirs(args.checkpoint_dir, exist_ok=True)
    os.makedirs("out", exist_ok=True)
    writer = SummaryWriter(args.logdir)

    # Data loaders
    ds = SyntheticRegressionDataset(
        num_samples=args.num_samples,
        input_size=args.input_size,
        val_split=args.val_split
    )
    train_loader, val_loader = ds.get_loaders(args.batch_size)

    # Model, scheduler, loss
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SelfRecursiveAGI(
        args.input_size, args.hidden_size, args.output_size,
        lr_main=args.lr_main, lr_self=args.lr_self,
        dropout_p=args.dropout_p
    ).to(device)
    scheduler = StepLR(model.main_opt, step_size=args.lr_step, gamma=args.lr_gamma)
    loss_fn = nn.MSELoss()

    global_step = 0
    for epoch in range(1, args.epochs + 1):
        model.train()
        epoch_loss = 0.0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{args.epochs}")
        for Xb, yb in pbar:
            Xb, yb = Xb.to(device), yb.to(device)

            # Outer-loop update
            model.main_opt.zero_grad()
            out    = model(Xb)
            l_main = loss_fn(out, yb)
            l_main.backward()
            nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            model.main_opt.step()

            # Inner-loop self-improvement
            l_self = model.self_improve(loss_fn, Xb, yb,
                                        steps=args.self_steps,
                                        clip=args.clip)

            epoch_loss += l_main.item()
            global_step += 1
            writer.add_scalar("train/loss_main", l_main.item(), global_step)
            writer.add_scalar("train/loss_self", l_self, global_step)

            # display on progress bar
            pbar.set_postfix(main=f"{l_main:.4f}", self_imp=f"{l_self:.4f}")

        scheduler.step()
        avg = epoch_loss / len(train_loader)
        print(f"→ Epoch {epoch} avg loss {avg:.6f}")

        if epoch % args.eval_interval == 0:
            # Save checkpoint
            ckpt = dict(
                epoch=epoch,
                model=model.state_dict(),
                optim=model.main_opt.state_dict(),
                sched=scheduler.state_dict()
            )
            ckpt_path = os.path.join(args.checkpoint_dir, f"agi_ep{epoch}.pt")
            torch.save(ckpt, ckpt_path)
            print(" checkpoint saved:", ckpt_path)

            # Evaluate & log
            evaluate_and_log(model, val_loader, device, writer,
                             epoch, args.mc_samples)

    writer.close()


# -----------------------------------------------------------------------------
# 4. Eval + Logging
# -----------------------------------------------------------------------------
def evaluate_and_log(model, val_loader, device, writer, epoch, mc_samples):
    model.eval()
    Y, P, S = [], [], []
    for Xb, yb in val_loader:
        xb   = Xb.to(device)
        mean, std = model.mc_predict(xb, mc_samples)
        Y.append(yb.numpy().squeeze())
        P.append(mean)
        S.append(std)

    y_true = np.concatenate(Y)
    y_pred = np.concatenate(P)
    uncert = np.concatenate(S)
    err    = np.abs(y_pred - y_true)

    # Scalars
    writer.add_scalar("val/mse", np.mean((y_pred - y_true)**2), epoch)
    writer.add_scalar("val/mae", np.mean(err), epoch)

    # Scatter plot
    fig, ax = plt.subplots(figsize=(4,4))
    ax.scatter(y_true, y_pred, s=5, alpha=0.5)
    mn, mx = y_true.min(), y_true.max()
    ax.plot([mn,mx],[mn,mx],'--', color='gray')
    ax.set_xlabel("True"); ax.set_ylabel("Pred")
    ax.set_title(f"Epoch {epoch} Pred vs True")
    writer.add_figure("val/scatter", fig, epoch)
    fig.savefig(f"out/epoch{epoch}_scatter.png")
    plt.close(fig)

    # Residual histogram
    fig, ax = plt.subplots()
    ax.hist(err, bins=30, alpha=0.7)
    ax.set_title(f"Epoch {epoch} Residuals")
    writer.add_figure("val/residuals", fig, epoch)
    fig.savefig(f"out/epoch{epoch}_resids.png")
    plt.close(fig)

    # Reliability diagram
    bins = np.linspace(uncert.min(), uncert.max(), 10)
    idx  = np.digitize(uncert, bins) - 1
    emp  = [err[idx==i].mean() if np.any(idx==i) else np.nan
            for i in range(len(bins)-1)]
    centers = 0.5*(bins[:-1] + bins[1:])
    fig, ax = plt.subplots()
    ax.plot(centers, emp, '-o', label="emp err")
    ax.plot([uncert.min(),uncert.max()],
            [uncert.min(),uncert.max()],
            '--', color='gray', label="ideal")
    ax.set_xlabel("σ_pred"); ax.set_ylabel("emp |err|")
    ax.set_title(f"Epoch {epoch} Reliability")
    ax.legend()
    writer.add_figure("val/reliability", fig, epoch)
    fig.savefig(f"out/epoch{epoch}_reliability.png")
    plt.close(fig)


# -----------------------------------------------------------------------------
# 5. Argument Parsing & Entry Point
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs",         type=int,   default=20)
    parser.add_argument("--batch_size",     type=int,   default=32)
    parser.add_argument("--input_size",     type=int,   default=10)
    parser.add_argument("--hidden_size",    type=int,   default=64)
    parser.add_argument("--output_size",    type=int,   default=1)
    parser.add_argument("--lr_main",        type=float, default=1e-3)
    parser.add_argument("--lr_self",        type=float, default=1e-3)
    parser.add_argument("--dropout_p",      type=float, default=0.1)
    parser.add_argument("--self_steps",     type=int,   default=3)
    parser.add_argument("--clip",           type=float, default=1.0)
    parser.add_argument("--lr_step",        type=int,   default=5)
    parser.add_argument("--lr_gamma",       type=float, default=0.5)
    parser.add_argument("--eval_interval",  type=int,   default=5)
    parser.add_argument("--mc_samples",     type=int,   default=50)
    parser.add_argument("--num_samples",    type=int,   default=2500)
    parser.add_argument("--val_split",      type=int,   default=500)
    parser.add_argument("--checkpoint_dir", type=str,   default="checkpoints")
    parser.add_argument("--logdir",         type=str,   default="runs/exp1")

    # ignore unknown args (e.g. Jupyter’s -f flag)
    args, _ = parser.parse_known_args()
    train_and_evaluate(args)